In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.oauth2 import service_account
from googleapiclient.discovery import build

# Switch off future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [8]:
# Link to the Google Sheet
link = 'https://docs.google.com/spreadsheets/d/1luO3H2QHzxrwnus9gsC8U3GMPLwwbG4py08Rv5IY1hs/edit?usp=sharing'

# Updated variables
spreadsheet_id = '1luO3H2QHzxrwnus9gsC8U3GMPLwwbG4py08Rv5IY1hs'
json_key_file = '/Users/timursalakhetdinov/Applied Statistical Analysis/Google_json/banded-anvil-309310-3642b97f7801.json'
sheet_name = 'Tabellenblatt1'  # Replace with the actual sheet name

# Authenticate using OAuth 2.0 JSON file
creds = service_account.Credentials.from_service_account_file(
    json_key_file,
    scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"]
)
service = build('sheets', 'v4', credentials=creds)

# Fetch data from the specified sheet
result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=sheet_name).execute()
values = result.get('values', [])

if values:
    # Convert to DataFrame
    df = pd.DataFrame(values[1:], columns=values[0])  # Assuming the first row is the header

    # Display the DataFrame
    display(df.head())
else:
    print("No data found in the sheet.")

,Name of the cookie,Nickname of the evaluator (anonymous),vegan? yes/no,no. of Ingredients,"Sweetness Factor (1, too sweet, 5 just perfect,10 not sweet enough)",Crunch Factor (1 too little crunch - 10 too much crunch (5just perfect)),"Crumble Factor (1, not crumble 10 tooooo much crumble)","Stickiness (0 no, 1 yes)","Appereance (1, is this a cookie? - 10 this cookie is art)","Contains chocolate (0 no, 1 yes)",...,color,"moisture (1=dry, 10=moist)",salty,nutty,"spiced (w/ Cinnamon, cloves, ginger etc), yes or no",Overall Score (1 to 10),coconut,Aroma,"Allergensm[flour, eggs, nuts etc.] yes or no",burnt
0,Jacobsen (black),Keks,,,3,6,5,0,9,1,...,Brown and green,4,,yes,No,7,None,None,None,None
1,X-MAS,Dr Cookie,yes,8,7,6,,0,8,1,...,brown,4,no,no,no,,no,no,yes,0
2,Jacobsen (black),CookieMonster,,,4,5,8,0,9,1,...,black & white,3,no,yes,no ?,9,no,no,no,0
3,I've tried my best,cookie regressor,yes,,5,2,,0,8,0,...,,8,no,no,no,9,yes,no,no,0
4,Chatgpt chocolate cookie,cookie_master,yes,7,7,2,4,0,7,1,...,brown,8,no,no,no,7,no,normal,no,0
